<center>
    <h1>Verbal Explanation of Spatial Temporal GNNs for Traffic Forecasting</h1>
    <h2>Verbal Explanations on the Metr-LA Dataset</h2>
</center>

---

The verbal translation occurs through a template-based approach. This method involves substituting placeholders in textual templates with the chosen content to form coherent narratives.

The verbal translation consists of composing a series of paragraphs by exploiting the content extracted from the graphical explanations. The first paragraph describes the predicted event and briefly sums up its causes, while the second to last paragraphs illustrate in detail each cause leading to the event which is each cluster of the important subgraph. The paragraphs describing the causes are sorted by the time they occurred.

In [1]:
import sys
import os

# Set the main path in the root folder of the project.
sys.path.append(os.path.join('..'))

In [2]:
# Settings for autoreloading.
%load_ext autoreload
%autoreload 2

In [3]:
from src.utils.seed import set_random_seed

# Set the random seed for deterministic operations.
SEED = 42
set_random_seed(SEED)

# 1 Loading the Data
In this section the data is loaded

In [5]:
import os

BASE_DATA_DIR = os.path.join('..', 'data', 'metr-la')

In [33]:
from src.data.data_extraction import get_adjacency_matrix

# Get the adjacency matrix
adj_matrix_structure = get_adjacency_matrix(
    os.path.join(BASE_DATA_DIR, 'raw', 'adj_mx_metr_la.pkl'))

# Get the header of the adjacency matrix, the node indices and the
# matrix itself.
_, node_ids_dict, _ = adj_matrix_structure

# Get the node positions dictionary.
node_pos_dict = { i: id for id, i in node_ids_dict.items() }

In [34]:
import pickle

# Get the node street and kilometrage dictionary.
with open(os.path.join(BASE_DATA_DIR, 'structured', 'node_locations.pkl'), 'rb') as f:
    node_info = pickle.load(f)

In [41]:
import os
import numpy as np

# Get the explained data.
x_train = np.load(os.path.join(BASE_DATA_DIR, 'explained', 'x_train.npy'))[..., :1]
y_train = np.load(os.path.join(BASE_DATA_DIR, 'explained', 'y_train.npy'))[..., :1]
x_val = np.load(os.path.join(BASE_DATA_DIR, 'explained', 'x_val.npy'))[..., :1]
y_val = np.load(os.path.join(BASE_DATA_DIR, 'explained', 'y_val.npy'))[..., :1]
x_test = np.load(os.path.join(BASE_DATA_DIR, 'explained', 'x_test.npy'))[..., :1]
y_test = np.load(os.path.join(BASE_DATA_DIR, 'explained', 'y_test.npy'))[..., :1]

# Get the clustered data.
x_train_clusters = np.load(os.path.join(BASE_DATA_DIR, 'clustered', 'x_train.npy'))
x_val_clusters = np.load(os.path.join(BASE_DATA_DIR, 'clustered', 'x_val.npy'))
x_test_clusters = np.load(os.path.join(BASE_DATA_DIR, 'clustered', 'x_test.npy'))


# Get the time information of the explained data.
x_train_times = np.load(os.path.join(BASE_DATA_DIR, 'explained', 'x_train_time.npy'))
y_train_times = np.load(os.path.join(BASE_DATA_DIR, 'explained', 'y_train_time.npy'))
x_val_times = np.load(os.path.join(BASE_DATA_DIR, 'explained', 'x_val_time.npy'))
y_val_times = np.load(os.path.join(BASE_DATA_DIR, 'explained', 'y_val_time.npy'))
x_test_times = np.load(os.path.join(BASE_DATA_DIR, 'explained', 'x_test_time.npy'))
y_test_times = np.load(os.path.join(BASE_DATA_DIR, 'explained', 'y_test_time.npy'))

The predictions are turned into km/h

In [42]:
from src.utils.config import MPH_TO_KMH_FACTOR

x_train = x_train * MPH_TO_KMH_FACTOR
y_train = y_train * MPH_TO_KMH_FACTOR

x_val = x_val * MPH_TO_KMH_FACTOR
y_val = y_val * MPH_TO_KMH_FACTOR

x_test = x_test * MPH_TO_KMH_FACTOR
y_test = y_test * MPH_TO_KMH_FACTOR

# 2 Verbal Translation
The translation is performed on the datasets

In [43]:
VERBAL_TRANSLATION_DIR = os.path.join(BASE_DATA_DIR, 'translated')

os.makedirs(VERBAL_TRANSLATION_DIR, exist_ok=True)

In [47]:
from src.verbal_explanations.verbal_translation import translate_dataset

train_translated = translate_dataset(
    x_train,
    x_train_times,
    x_train_clusters,
    y_train,
    y_train_times,
    node_pos_dict,
    node_info)
    

np.save(os.path.join(VERBAL_TRANSLATION_DIR, 'train.npy'), train_translated)

val_translated = translate_dataset(
    x_val,
    x_val_times,
    x_val_clusters,
    y_val,
    y_val_times,
    node_pos_dict,
    node_info)

np.save(os.path.join(VERBAL_TRANSLATION_DIR, 'val.npy'), val_translated)

test_translated = translate_dataset(
    x_test,
    x_test_times,
    x_test_clusters,
    y_test,
    y_test_times,
    node_pos_dict,
    node_info)

np.save(os.path.join(VERBAL_TRANSLATION_DIR, 'test.npy'), test_translated)

Following an example of a verbal translation

In [50]:
print(test_translated[0])

San diego freeway at kms 11, 12, 13 and 14 was forecasted to see severe congestion on Wednesday, 04/06/2012, with an average speed of 24.04 km/h from 07:45 to 08:40. This occurred because of a series of congestions and a free flow.

To start, a contributing free flow materialized, at 107.75 km/h, on San Diego Freeway at kms 8 and 10 from 06:45 to 06:50.

Following this, a contributing congestion occurred, averaging at a speed of 82.83 km/h, on, another time, San Diego Freeway at kms 9 and 10 from 06:45 to 07:10.

Afterwards, a new contributing congestion took place, with an average speed of 88.65 km/h, on Ventura Freeway at kms 27, 28, 29, 30 and 31 occurring from 06:45 to 07:40. The congestion also took place on, once again, San Diego Freeway at kms 11, 12 and 13.

After this, a contributing severe congestion happened on, yet again, San Diego Freeway at kms 10, 11, 12, 13, 14 and 15 from 06:45 to 07:40 with an average speed of 19.70 km/h.

To conclude, yet a further contributing conge